In [1]:
import csv
import enum
import datetime

In [2]:
class enmWeatherComponent(enum.IntEnum):
    Ta = 1     # 外気温度[℃]
    x = 5      # 絶対湿度[g/kg']
    Idn = 2    # 法線面直辰日射量[W/m2]
    Isky = 3   # 水平面天空日射量[W/m2]
    RN = 4     # 夜間放射量[W/m2]
    Ihol = 6   # 水平面全天日射量[W/m2]
    h = 7      # 太陽高度[rad]
    A = 8      # 太陽方位角[rad]

In [3]:
# 毎時気象データの所得
class Weather:

    # 気象データの取得
    def __init__(self, Lat, Lon, Ls):

        # 太陽位置計算クラスの作成
        self.__objSolpos = SolarPosision(Lat, Lon, Ls)
        
        # 気象データの読み込み　→　dblWdata
        with open('気象データ.csv') as f:
            reader = csv.reader(f)
            header = next(reader)
            header = next(reader)
            day = 1
            time = 1
            # データの形がVBAと違うことに注意！
            # VBAでは、dbl[5][365][24]の3次元配列。(VBなので1始まりであることに注意。)
            # ここでは、列1にday,列2にtime、列3～列7に5データを持つ2次元配列であることに注意。
            self.__dblWdata = []
            for row in reader:
                # データ自身は全部で7項目(温度、法線面直達日射量、水平面全天日射量、夜間放射量、絶対湿度)
                # このうち、VBAでは、夜間放射量と絶対湿度は捨てているので、VBAどおり、5データのみデータ化する。
                # time は、1始まりであることに注意。
                self.__dblWdata.append([day,time,float(row[2]),float(row[3]),float(row[4]),float(row[5]),float(row[6])])
                time = time + 1
                if time > 24 :
                    time = 1
                    day = day + 1
            
    # 気象データの取得
    # 戻り値はdouble
    def WeaData(Compnt, dtmDate, blnLinear = True):
        # Compnt:取得する気象要素
        # dtmDate:取得する日時
        # blnLinear:線形補間するかどうか（Trueは線形補間する）
        
        # 通日、時、分、秒の取得
        lngNday, lngHour = Address(dtmDate)
        lngMinu = dtmDate.minute
        lngSec = dtmDate.second
        
        # 線形補間時の案分比
        dblR = lngMinu / 60 + lngSec / 3600
        
        # 1時間後のアドレスを取得
        lngNday2, lngHour2 = Address(dtmDate + datetime.timedelta(hour = 1) )

        # 太陽位置の取得
        
    

In [4]:
# Date型日時から取得する通日、時刻アドレスを計算する
# 0時を24時に変換するため
# 補間は行わない
def Address(dtmDate):
    # dtmDate は、datetime オブジェクトとする。
    lngHour = dtmDate.hour
    # 1月1日から数えて何日目にあたるのかを計算する。
    lngNday = (datetime.date(dtmDate.year, dtmDate.month, dtmDate.day) - datetime.date(dtmDate.year, 1, 1)).days + 1
    if lngHour == 0:
        lngNday = lngNday - 1
        lngHour = 24    
    if lngNday == 0:
        lngNday = 365
    return (lngNday, lngHour)

In [5]:
Address(datetime.datetime.strptime('2017/01/01 12:06:19', '%Y/%m/%d %H:%M:%S'))

(1, 12)

In [6]:
Address(datetime.datetime.strptime('2017/01/01 00:06:19', '%Y/%m/%d %H:%M:%S'))

(365, 24)

In [7]:
Address(datetime.datetime.strptime('2017/01/04 00:06:19', '%Y/%m/%d %H:%M:%S'))

(3, 24)